IMPORT OF USED LIBRARIES

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

LOADED OUTPUT DATASET

In [ ]:
path_to_raw_output_data = "data\RozsAgloSP4-2018-vystup.xlsm"

data_output = pd.read_excel(path_to_raw_output_data)

data_output

In [ ]:
adjusted_times = []

# Loop through each time stamp in the 'Dátum a čas' column of data_output
for time_stamp in data_output['Dátum a čas']:

    # Convert time stamp to string and then to datetime object
    time_str = time_stamp.strftime("%Y-%m-%d %H:%M:%S")
    date_time = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")

    # Adjust the time based on conditions
    if date_time.hour >= 23:
        date_time = date_time.replace(hour=23, minute=0, second=0)
    elif date_time.hour >= 11: 
        date_time = date_time.replace(hour=11, minute=0, second=0)
    else:
        date_time -= timedelta(days=1)  # Subtract a day
        date_time = date_time.replace(hour=23, minute=0, second=0)

    adjusted_times.append(date_time)  # Add adjusted time to list

data_output['Upravené časy intervalov'] = adjusted_times  # Add adjusted times to data_output

closest_time_stamps = []
groups = data_output.groupby('Upravené časy intervalov')

# Loop through groups
for _, group in groups:

    # Get first time stamp in the group
    first_time_stamp = group.iloc[0]['Dátum a čas']
    closest_time_stamp = first_time_stamp

    # Iterate through rows in the group
    for _, row in group.iterrows():

        real_time_stamp = row['Dátum a čas']

        # Check if the absolute time difference is smaller than the current closest time stamp
        if abs((real_time_stamp - row['Upravené časy intervalov']).total_seconds()) < abs((closest_time_stamp - row['Upravené časy intervalov']).total_seconds()):
            closest_time_stamp = real_time_stamp
    
    closest_time_stamps.append(closest_time_stamp)

# Filter data_output to keep rows with closest_time_stamps
data_w_relevant_ts = data_output[data_output['Dátum a čas'].isin(closest_time_stamps)]
data_w_relevant_ts = data_w_relevant_ts.reset_index(drop=True)

# Remove specified columns with NaN values
data_output = data_output.drop(['ISO Zhoda', 'ISO 0-5', 'ISO > 6,3'], axis=1)

In [ ]:
path_to_15min_raw_data = 'data\PI_DataLink_Aglo_digital twins-5min-SP4-2018-15minute.xlsm'

data_15min = pd.read_excel(path_to_15min_raw_data)

data_15min

In [ ]:
path_to_30min_raw_data = 'data\PI_DataLink_Aglo_digital twins-5min-SP4-2018-30minute.xlsm'

data_30min = pd.read_excel(path_to_30min_raw_data)

data_30min

REMOVAL OF REDUNDANT DATA

In [ ]:
data_15min = data_15min.iloc[3:, 2:]
data_15min = data_15min.reset_index(drop=True)

data_30min = data_30min.iloc[3:, 2:]
data_30min = data_30min.reset_index(drop=True)

# Drop rows with NaN values
data_15min.dropna(inplace=True)

data_30min.dropna(inplace=True)

MERGING BASED ON DATE AND TIME


- For 15 minutes data

In [ ]:
# Convert date columns to datetime format
data_output['Date'] = pd.to_datetime(data_output['Upravené časy intervalov'])
data_15min['Date'] = pd.to_datetime(data_15min['Unnamed: 2'])

# Merge based on date intervals
result_data = pd.merge(data_15min, data_output, left_on=['Date'], right_on=['Date'], how='inner')

# Rename columns
result_data = result_data.rename(columns={'Unnamed: 2': 'Interval od', 'Unnamed: 3': 'Interval do'})

# Drop unnecessary columns
result_data = result_data.drop('Date', axis=1)  # Drop the merged date column
result_data = result_data.drop('výstupné dáta, látkové filtre - pred komínom', axis=1)  # Drop specific columns
result_data = result_data.drop('Unnamed: 12', axis=1)
result_data = result_data.drop('Unnamed: 13', axis=1)
result_data = result_data.drop('Unnamed: 14', axis=1)
result_data = result_data.drop('Unnamed: 16', axis=1)
result_data = result_data.drop('výstupné dáta.3', axis=1)
result_data = result_data.drop('výstupné dáta.4', axis=1)
result_data = result_data.drop('Unnamed: 27', axis=1)
result_data = result_data.drop('vstupné dáta, reguluje palič.10', axis=1)
result_data = result_data.drop('Unnamed: 30', axis=1)
result_data = result_data.drop('vstupné dáta.2', axis=1)
result_data = result_data.drop('vstupné dáta.3', axis=1)
result_data = result_data.drop('vstupné dáta, 3 stupeň', axis=1)

# Save to Excel
result_15min_data_path = 'data/parsed_15min_dataset.xlsx'
result_data.to_excel(result_15min_data_path, index=False)

In [ ]:
result_data.count()

- for 30 minutes data

In [ ]:
# Convert date columns to datetime format

data_30min['Date'] = pd.to_datetime(data_30min['Unnamed: 2'])

# Merge based on date intervals
result_data = pd.merge(data_30min, data_output, left_on=['Date'], right_on=['Date'], how='inner')

# Rename columns
result_data = result_data.rename(columns={'Unnamed: 2': 'Interval od', 'Unnamed: 3': 'Interval do'})

# Drop unnecessary columns
result_data = result_data.drop('Date', axis=1)  # Drop the merged date column
result_data = result_data.drop('výstupné dáta, látkové filtre - pred komínom', axis=1)  # Drop specific columns
result_data = result_data.drop('Unnamed: 12', axis=1)
result_data = result_data.drop('Unnamed: 13', axis=1)
result_data = result_data.drop('Unnamed: 14', axis=1)
result_data = result_data.drop('Unnamed: 16', axis=1)
result_data = result_data.drop('výstupné dáta.3', axis=1)
result_data = result_data.drop('výstupné dáta.4', axis=1)
result_data = result_data.drop('Unnamed: 27', axis=1)
result_data = result_data.drop('vstupné dáta, reguluje palič.10', axis=1)
result_data = result_data.drop('Unnamed: 30', axis=1)
result_data = result_data.drop('vstupné dáta.2', axis=1)
result_data = result_data.drop('vstupné dáta.3', axis=1)
result_data = result_data.drop('vstupné dáta, 3 stupeň', axis=1)

# Save to Excel
result_30min_data_path = 'data/parsed_30min_dataset.xlsx'
result_data.to_excel(result_30min_data_path, index=False)

In [ ]:
result_data.count()

ADJUSTING RESULT DATASET TO INFO DATASET

In [ ]:
# Read the parsed_dataset excel file and select specific columns
info_dataset = pd.read_excel(result_15min_data_path)
info_dataset = info_dataset[['Interval od', 'Interval do', 'Vzorka', 'Zhoda', '0-5', 'výstupné dáta.5']]

# Save the selected columns to a new excel file
info_dataset.to_excel('data/info_15min_dataset.xlsx', index=False)

# Read the parsed_dataset excel file and select specific columns
info_dataset = pd.read_excel(result_30min_data_path)
info_dataset = info_dataset[['Interval od', 'Interval do', 'Vzorka', 'Zhoda', '0-5', 'výstupné dáta.5']]

# Save the selected columns to a new excel file
info_dataset.to_excel('data/info_30min_dataset.xlsx', index=False)

In [ ]:
info_dataset.count()

ADJUSTING RESULT DATASET TO FINAL DATASET FOR NEURAL NETWORK

In [ ]:
final_dataset = result_data.drop(columns=['Interval od', 'Interval do', 'Vzorka', 'Zhoda', 'výstupné dáta.5', 'Upravené časy intervalov'])

final_dataset.drop_duplicates(subset='0-5')
final_dataset.dropna(inplace=True)
final_dataset.reset_index(drop=True)

final_dataset_path = 'data/15min_dataset_pre_NN.xlsx'
final_dataset.to_excel(final_dataset_path, index=False)

In [ ]:
final_dataset